In [153]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error
import pandas as pd

In [154]:
request_call_procedures_ManageBooking = """ CALL capstone_module4.ManageBooking();"""
request_call_procedures_AddBooking = """ CALL capstone_module4.AddBooking();"""
request_call_procedures_UpdateBooking = """ CALL capstone_module4.UpdateBooking();"""
request_call_procedures_CancelBooking = """ CALL capstone_module4.CancelBooking();"""

In [ ]:

dbconfig = {
    "user": "user1",
    "password": "*****",
    "host": "localhost",
    "database": "little_lemon_db"
}

# Create the connection pool
def create_pool(pool_size, pool_name): 
    try:
        pool = MySQLConnectionPool(pool_name = pool_name,
                            pool_size = pool_size,
                            **dbconfig)
        print("The connection pool is created with a name: ",pool.pool_name)
        print("The pool size is:",pool.pool_size)
        return pool

    except Error as er:
        print("Error code:", er.errno)
        print("Error message:", er.msg)

def query_db(sql_query='', file_path='', pool='', fetch=True, type='query'): 
    try: 
        connection = pool.get_connection()
        cursor = connection.cursor()
        if type == 'query': 
            cursor.execute(sql_query)
        elif type == 'sql_file': 
                # Read and execute the SQL file
            with open(file_path, "r", encoding="utf-8") as file:
                sql_script = file.read()

            cursor.execute(sql_script)
            
        elif type == 'csv_file':
            # Fix column name handling
            table_name = "data" 
            # Load CSV file with auto separator detection
            df = pd.read_csv(file_path, sep=None, engine="python")

            # ✅ Fix: Clean and format column names
            cleaned_columns = [col.rstrip().replace(",", "_") for col in df.columns]
            cleaned_columns = [f"{col}_{i}" if cleaned_columns.count(col) > 1 else col for i, col in enumerate(cleaned_columns)]
            columns_sql = ", ".join([f"`{col}` VARCHAR(255)" for col in cleaned_columns])

            create_table_query = f"""
            CREATE TABLE IF NOT EXISTS capstone_module4.`{table_name}` (
                id INT AUTO_INCREMENT PRIMARY KEY,
                {columns_sql}
            );
            """
            cursor.execute(create_table_query)  # Create the table
            connection.commit()

            # ✅ Corrected INSERT Query (No extra loops)
            insert_query = f"""
            INSERT INTO capstone_module4.`{table_name}` ({', '.join([f'`{col}`' for col in df.columns])}) 
            VALUES ({', '.join(['%s' for _ in df.columns])});
            """

            # ✅ Insert data row by row
            for _, row in df.iterrows():
                cursor.execute(insert_query, tuple(row))

            connection.commit()  # Save changes

        result = None
        if fetch and cursor.with_rows:  # ✅ Vérifie si la requête retourne un résultat
            result = cursor.fetchall()
        
        while cursor.nextset():  # 🔥 Vide tous les résultats restants
            pass

        connection.commit()
        return result

    except Exception as e:
        print(f"Error occurred: {e}")
        if connection.is_connected():
            connection.rollback()

    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'connection' in locals() and connection.is_connected():
            connection.close()
            


In [ ]:
# Create pool (size 3)
first_pool = create_pool(3, "first_pool")
# Définition des fichiers et paramètres sous forme de dictionnaire
queries = {
    "request_drop_schema": ("SQL/DROP_SCHEMA.sql", False, "sql_file"),
    "query_db_from_SQLfile": ("SQL/CREATE.sql", False, "sql_file"),
    "query_db_from_CSVfile": ("data.csv", False, "csv_file"),
    "request_populate_tables": ("SQL/INSERT.sql", False, "sql_file"),
    "request_create_procedures_AddBooking": ("SQL/Create_Procedure_AddBooking.sql", False, "sql_file"),
    "request_create_procedures_CancelBooking": ("SQL/Create_Procedure_CancelBooking.sql", False, "sql_file"),
    "request_create_procedures_ManageBooking": ("SQL/Create_Procedure_ManageBooking.sql", False, "sql_file"),
    "request_create_procedures_UpdateBooking": ("SQL/Create_Procedure_UpdateBooking.sql", False, "sql_file"),
}

# Exécution des requêtes de création et d’insertion
for name, (file_path, fetch, file_type) in queries.items():
    query_db(file_path=file_path, pool=first_pool, fetch=fetch, type=file_type)
    print(f"{name} ok")

# Dictionnaire pour les appels de procédures stockées
procedures = {
    "request_call_procedures_ManageBooking": request_call_procedures_ManageBooking,
    "request_call_procedures_AddBooking": request_call_procedures_AddBooking,
    "request_call_procedures_UpdateBooking": request_call_procedures_UpdateBooking,
    "request_call_procedures_CancelBooking": request_call_procedures_CancelBooking,
}

# Exécution des procédures stockées
for name, procedure in procedures.items():
    query_db(procedure, pool=first_pool, fetch=True, type='query')
    print(f"{name} ok")






The connection pool is created with a name:  first_pool
The pool size is: 3
request_drop_schema ok
query_db_from_SQLfile ok
query_db_from_CSVfile ok
request_populate_tables ok
request_create_procedures_AddBooking ok
request_create_procedures_CancelBooking ok
request_create_procedures_ManageBooking ok
request_create_procedures_UpdateBooking ok
request_call_procedures_ManageBooking ok
request_call_procedures_AddBooking ok
request_call_procedures_UpdateBooking ok
request_call_procedures_CancelBooking ok
